In [29]:
# Import BeautifulSoup, Splinter, Chromedriver
from bs4 import BeautifulSoup
from splinter import Browser
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
import time
executable_path = {"executable_path": "chromedriver"}

# Step 1 - Scraping

Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.
Create a Jupyter Notebook file called mission_to_mars.ipynb and use this to complete all of your scraping and analysis tasks. The following outlines what you need to scrape.

## Creating a dictionary to fill with the extracted data from several websites

In [25]:
data={}

### NASA Mars News
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [32]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

options=Options()
options.add_argument('--ignore-certificate-errors')
browser = Browser("chrome", **executable_path, headless=False,chrome_options=options)
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


In [33]:
NASA_Mars_news={}
for item_list in soup.find_all('li',class_='slide'):
    item=item_list.find('div',class_='image_and_description_container')
    news_title=item.find('div', class_='content_title').text
    news_p=item.find('div', class_='article_teaser_body').text
    print(news_title)
    print(news_p)
    print()
    NASA_Mars_news.update({news_title:news_p})
data.update({'NASA_MARS_NEWS':NASA_Mars_news})

### JPL Mars Space Images - Featured Image
Visit the url for JPL Featured Space Image here.
Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

In [34]:
#browser = Browser("chrome", **executable_path, headless=False)
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(3)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
temp = soup.find('div', {'id':'main_container'}).\
         find('div', {'id':'site_body'}).find('div', {'id':'page'}).\
        find('div', class_='carousel_container')

url_part=temp.find('article')['style'].split('/',1)[1][:-3]
featured_image_url="https://www.jpl.nasa.gov/" + url_part

print(featured_image_url)
data.update({'JPL_MARS_SPACE_IMAGE':featured_image_url})

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19871-1920x1200.jpg


### Mars Weather
Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.

In [35]:
#browser = Browser("chrome", **executable_path, headless=False)
url='https://twitter.com/marswxreport?lang=en'
browser.visit(url)  
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
mars_weather=soup.find('div',{'id':'doc'}).find('div',{'id':'timeline'}).\
        find('div',class_='stream').find('div',class_='content').p.text
print(mars_weather)
data.update({'MARS_WEATHER':mars_weather})

Sunrise and sunset @NASAInSight sol 145, 5:30am and 6:30pm local time https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA23201 …pic.twitter.com/GfLOt2KOWU


### Mars Facts
Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
Use Pandas to convert the data to a HTML table string.

In [36]:
request_mars_space_facts = requests.get("https://space-facts.com/mars/")
mars_facts= pd.read_html(request_mars_space_facts.text)[0]
mars_facts.columns=['Facts', 'values']
mars_facts.set_index('Facts',inplace=True)
mars_facts=mars_facts.to_html(index='False',index_names='False')
data.update({'MARS_FACTS':mars_facts})

### Mars Hemispheres

Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title. Append the dictionary with the image url string and the hemisphere title to a list. 

In [38]:
#browser = Browser("chrome", **executable_path, headless=False)
url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)  
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
base_url='https://astrogeology.usgs.gov/'

First, we find the hemisphere titles and then retrieve the full image from each webpage using the title

#### Titles:

In [39]:
collapse=soup.find('div', class_='collapsible results')
hemisphere_titles=[]
for no,item in enumerate(collapse.find_all('div',class_='item')):
    print(item.a['href'].split('/')[-1])
    hemisphere_titles.append(item.a['href'].split('/')[-1])

cerberus_enhanced
schiaparelli_enhanced
syrtis_major_enhanced
valles_marineris_enhanced


#### Images + Dictionary

In [40]:
img_base_url='https://astrogeology.usgs.gov/search/map/Mars/Viking/'
hemisphere_image_urls ={}
for no,tk in enumerate(hemisphere_titles):
    #browser = Browser("chrome", **executable_path, headless=False)
    browser.visit(img_base_url+tk)
    time.sleep(1)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    hemisphere_image_urls.update({f'title{no}':tk,
                                  f'img_url{no}':soup.find('div',class_='container').\
                                  find('div',class_='downloads').ul.li.a['href']})
    print(soup.find('div',class_='container').\
                                  find('div',class_='downloads').ul.li.a['href'])
    
data.update({'MARS_HEMISPHERES':hemisphere_image_urls})

http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [ ]:
data